# 基本功能

pandas中的对象包含多种方法。

首先导入pandas相关的模块。

In [2]:
import pandas as pd
import numpy as np

## reindex

pandas中的数据结构具有的一个重要方法即reindex，该方法会接受一个list作为参数，用于创建一个与原对象相同的新对象，并将新对象的index设置为给定的list参数，

In [3]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

reindex的method属性为ffill时，指定了对有序index列表中缺省项采用依次填补的方式补充完整。若不指定method属性，则新对象的index列表的缺省向为NaN。

In [4]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [5]:
obj3.reindex(range(6))

0      blue
1       NaN
2    purple
3       NaN
4    yellow
5       NaN
dtype: object

对于DataFrame类型，reindex方法可以更改index、columns，或者两者同时修改。

我们还可以通过loc方法替代reindex方法，更加简洁的完成同样的功能**（官方已经不推荐这一做法，该方法存在安全隐患）**。

In [6]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [7]:
states = ['Texas', 'Utah', 'California']
frame2 = frame.reindex(['a', 'b', 'c', 'd'], columns=states)
frame2

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


|参数|备注|
|-|-|
|index||
|method||
|fill_value||
|limit||
|tolerance||
|level||
|copy||

## drop

对于Series对象，drop方法会删除对象中一行或多行，并返回一个删除行后的新对象，原对象不受影响。

In [9]:
# 删除单行
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [ ]:
# 删除多行
obj.drop(['d', 'c'])

对于DataFrame对象，drop方法既可以根据index删除行，也可以根据columns删除列。在未指定axis属性值时，drop方法行为与Series对象中类似，默认删除DataFrame对象的行，当axis值为1或columns时，drop删除DataFrame的指定列。

In [11]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [12]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [13]:
data.drop('two', axis='columns')

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15
